In [2]:
import pandas as pd
import importlib

In [3]:
import basic_example2 as be
importlib.reload(be)
import plot
importlib.reload(plot)

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


<module 'plot' from '/home/julia/school/bakalarka/hospital_sim/plot.py'>

In [4]:
def simulate(nurse_amount: int=3, patient_amount: int=5, requests: list[tuple[int, int]]=None) -> tuple[pd.DataFrame, plot.Plot]:
    nurses, patients = be.create_nurses_and_patients(nurseAmount=nurse_amount, patientAmount=patient_amount)
    if requests == None:
        requests = be.generate_requests(len(patients))
    patient_dsts = []
    patient_nurses = []
    for patient in patients:
        patient_dsts.append(patient.office_dst)
        patient_nurses.append(patient.nurse_id)
    sim = be.Simulator(requests, nurses, patients)
    df = pd.DataFrame(sim.simulate())
    outDict = df.to_dict(orient='records') 
    my_plot = plot.Plot(outDict, nurse_amount, patient_amount, patient_dsts, patient_nurses)
    return df, my_plot

In [ ]:
%%capture
df, my_plot = simulate(3, 3, [(1, 0), (2, 1), (3, 1), (4, 0)]) 
my_plot.run()

In [9]:
%%capture
df, my_plot = simulate(3, 5) 
my_plot.run()

SystemExit: 

In [10]:
%%capture
df, my_plot = simulate(3, 3, [(1, 0), (25, 2), (26, 1), (50, 0)]) 
my_plot.run()

SystemExit: 

In [ ]:
#kolikrát sestra vyřizovala pacienta
def patients_per_nurse(nurse_id):
    nurse_df =  df[df["chosen_nurse_id"] == nurse_id]
    display(nurse_df["patient_id"].value_counts())

In [ ]:
patients_per_nurse(1)

In [ ]:
#kolik toho sestra nachodila
def nurse_distance_walked(nurse_id):
    nurse_df = df[df["nurse_id"] == nurse_id]
    times = nurse_df["time"].to_list()
    assert(len(times) % 4 == 0)
    request_num = (len(times) / 4)
    walk_time_there = times[1] - times[0]
    walk_time_back = times[3] - times[2]
    walk_time = request_num * (walk_time_there + walk_time_back)
    # display(nurse_df)
    return walk_time

print(nurse_distance_walked(0))

In [ ]:
#jak dlouho pacient čekal
def total_wait_time(patient_id):
    patient_df = df[df["patient_id"] == patient_id]
    display(patient_df)
    reqs = patient_df["request_id"].unique()
    total_time = 0
    for request in reqs:
        req_df = patient_df[patient_df["request_id"] == request]
        # display(req_df)
        start_time = req_df.iloc[0].time
        end_time = req_df[req_df["nurse_phase"] == "arrived"].iloc[0]["time"] #surely this can be done more elegantly
        # end_time = req_df.iloc[-3]["time"] would work here but not in the future
        # print(end_time)
        time_waiting = end_time - start_time
        # print(start_time, end_time, time_waiting)
        total_time += time_waiting
    return total_time

print(total_wait_time(1))
# reqs

In [ ]:
len(df[df["patient_id"] == 1]["request_id"].unique())

In [ ]:
nurseAmount = 5
patientAmount = 10
simulations = 2
nurses, patients = be.create_nurses_and_patients(nurseAmount=nurseAmount, patientAmount=patientAmount)
walk_times = [0] * nurseAmount

for _ in range(simulations):
    requests = be.generate_requests(len(patients))
    sim = be.Simulator(requests, nurses, patients)
    df = pd.DataFrame(sim.simulate())

    for n in range(nurseAmount):
        walk_times[n] += nurse_distance_walked(n)

avg_walk_times = list(map(lambda x : x / simulations, walk_times))
print(avg_walk_times)

In [ ]:
nurses, patients = be.create_nurses_and_patients(nurseAmount=3, patientAmount=3)
requests = [(1, 0), (25, 2), (26, 1), (50, 0)] #time, patient id
sim = be.Simulator(requests, nurses, patients)
out = sim.simulate()
df2 = pd.DataFrame(out) #TODO design some structure to not have to go through df to keep NaNs
outDict = df2.to_dict(orient='records') 
simPlot = plot.Plot(outDict)
simPlot.run()

In [ ]:
outDict = df2.to_dict(orient='records')
print(outDict)